In [1]:
# %% [code]
# Instalación de dependencias necesarias
!pip install transformers gTTS diffusers torch moviepy whisper gradio requests



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 10.1 MB/s eta 0:00:00
   

In [ ]:
# %% [code]
# Instalación de dependencias necesarias
!pip install transformers diffusers gTTS moviepy gradio
!pip install git+https://github.com/openai/whisper.git

# %% [code]
# 1. Generación de guion con un modelo GPT-2 en español
from transformers import AutoModelForCausalLM, AutoTokenizer

modelo_nombre = "datificate/gpt2-small-spanish"  # Modelo gratuito en español
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
model = AutoModelForCausalLM.from_pretrained(modelo_nombre)

def generar_guion(tema, max_length=200):
    prompt = f"Escribe un guion explicativo sobre {tema} en tono educativo."
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, do_sample=True, temperature=0.7)
    guion = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return guion

def generar_guion_de_tema(tema):
    return generar_guion(tema)

# %% [code]
# 2. Conversión de guion a voz con gTTS
from gtts import gTTS

def generar_audio(texto, archivo_salida="audio.mp3"):
    tts = gTTS(texto, lang="es")
    tts.save(archivo_salida)
    return archivo_salida

# %% [code]
# 3. Generación de imagen con Stable Diffusion
import torch
from diffusers import StableDiffusionPipeline

# Se recomienda usar GPU; asegúrate de habilitarla en Colab.
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.to("cuda")

def generar_imagen(prompt, archivo_salida="imagen.png"):
    imagen = pipe(prompt).images[0]
    imagen.save(archivo_salida)
    return archivo_salida

# %% [code]
# 4. Edición y ensamblaje del video con MoviePy
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips

def crear_video(imagenes, audio, salida="video.mp4", duracion_por_imagen=5):
    clips = [ImageClip(img).set_duration(duracion_por_imagen) for img in imagenes]
    video = concatenate_videoclips(clips, method="compose")
    audio_clip = AudioFileClip(audio)
    video = video.set_audio(audio_clip)
    video.write_videofile(salida, fps=24)
    return salida

# %% [code]
# 5. Generación de subtítulos automáticos con Whisper
import whisper  # Importar la versión oficial de Whisper

model_whisper = whisper.load_model("small")  # Carga del modelo pequeño para prototipos

def generar_subtitulos(audio_file, subtitulo_salida="subtitulos.srt"):
    resultado = model_whisper.transcribe(audio_file)
    with open(subtitulo_salida, "w", encoding="utf-8") as f:
        f.write(resultado["text"])
    return subtitulo_salida

# %% [code]
# 6. (No anda) Narrador Virtual con Wav2Lip
# Nota: Este módulo requiere clonar el repositorio y contar con el checkpoint adecuado.
def aplicar_wav2lip(video_base, audio, salida="video_narrador.mp4", checkpoint_path="Wav2Lip/checkpoint/wav2lip.pth"):
    !git clone https://github.com/Rudrabha/Wav2Lip.git
    !python Wav2Lip/inference.py --checkpoint_path {checkpoint_path} --face {video_base} --audio {audio} --outfile {salida}
    return salida

# %% [code]
# 7. Función para generar el video a partir de un guion editado
from moviepy.editor import AudioFileClip

def generar_video_con_guion(guion):
    # Paso 2: Generar audio a partir del guion proporcionado (ya editado)
    archivo_audio = "audio.mp3"
    generar_audio(guion, archivo_salida=archivo_audio)

    # Obtener la duración total del audio para sincronizar las imágenes
    audio_clip = AudioFileClip(archivo_audio)
    audio_duration = audio_clip.duration

    # Paso 3: Dividir el guion en oraciones (separadas por punto)
    oraciones = [oracion.strip() for oracion in guion.split('.') if oracion.strip()]
    num_oraciones = len(oraciones)

    # Calcular duración para cada imagen (aproximada)
    duracion_por_imagen = audio_duration / num_oraciones if num_oraciones > 0 else 5

    # Paso 4: Generar una imagen para cada oración
    imagenes = []
    for i, oracion in enumerate(oraciones):
        # Se añade "sin texto" para evitar que aparezca texto en la imagen
        prompt_imagen = f"Ilustración digital art que muestra: {oracion} sin texto"
        archivo_imagen = f"imagen_{i}.png"
        generar_imagen(prompt_imagen, archivo_salida=archivo_imagen)
        imagenes.append(archivo_imagen)

    # Paso 5: Crear el video combinando las imágenes y el audio
    archivo_video = "video.mp4"
    crear_video(imagenes, archivo_audio, salida=archivo_video, duracion_por_imagen=duracion_por_imagen)

    # Paso 6: Generar subtítulos automáticos (archivo SRT)
    archivo_subtitulos = generar_subtitulos(archivo_audio)

    return archivo_video, archivo_subtitulos, guion

# %% [code]
# 8. Interfaz final con Gradio (dos pasos: generar/editar guion y generar video)
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Generador Automático de Videos Explicativos")
    gr.Markdown("Primero, ingresa un tema para generar el guion. Luego, edítalo si lo deseas y genera el video final.")

    with gr.Tab("Generar y Editar Guion"):
        with gr.Row():
            tema_input = gr.Textbox(label="Tema del Video", placeholder="Ej: La historia del internet")
            btn_guion = gr.Button("Generar Guion")
        guion_text = gr.Textbox(label="Guion Generado (editable)", lines=10)
        btn_guion.click(fn=generar_guion_de_tema, inputs=tema_input, outputs=guion_text)

    with gr.Tab("Generar Video"):
        guion_editable = gr.Textbox(label="Guion Editable", lines=10)
        btn_video = gr.Button("Generar Video")
        video_output = gr.File(label="Video Explicativo")
        subtitles_output = gr.File(label="Subtítulos (.srt)")
        guion_final = gr.Textbox(label="Guion Final")
        btn_video.click(fn=generar_video_con_guion, inputs=guion_editable, outputs=[video_output, subtitles_output, guion_final])

demo.launch()


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-mexmeigu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-mexmeigu
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

(…)ure_extractor%2Fpreprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  if event.key is 'enter':

100%|███████████████████████████████████████| 461M/461M [00:05<00:00, 90.3MiB/s]
  checkpoint = torch.load(fp, map_location=device)



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f954ca7363a97060de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
